In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
#chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://iafor.org/conferences/'

driver.get(url)
time.sleep(5)

soup = bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':re.compile('conference-card post type-post.+')})
len(cards)

sanp=[]
for m in cards:
# for k in range(1,22):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
    tit=m.h2.text
    link=m.find('a')['href']
    datloc=m.find('div',{'class':'conference-card-dates-and-location'}).text
    dat,loc=[a.strip() for a in datloc.split('|')]
    sanp.append([tit,link,datloc,dat,loc])


In [19]:
iafor=[]
for m in cards:
# for k in range(1,22):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
    title=m.h2.text
    link=m.find('a')['href']
    datloc=m.find('div',{'class':'conference-card-dates-and-location'}).text
    #driver.get(link)
    time.sleep(5)
    #soupn=bs(driver.page_source,'lxml')
    dat,loc=[a.strip() for a in datloc.split('|')]
    for m in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
        if m in dat:
            dat=dat.replace(m,'')
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').strip().replace('Oc-ber','October').replace('OC-BER','OCTOBER')
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
        #('September 8, 2022 I The Ritz-Carlton, Beijing',
    if re.search('[A-Sa-z]{3,9}\s?\d{1,2}\s+\d{4}\s?[A-Za-z]+\s?[A-Za-z]+\s?[A-Za-z]+',mad.replace(',','')):
        #print('juice')
        tad=re.search('[A-Sa-z]{3,9}\s?\d{1,2}\s+\d{4}',mad.replace(',','')).group()
        mad=tad
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        #print(mad)
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')


    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v.strip(), '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v.strip(), '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]   

    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            sha=location.split(',')
            try:
                gu(sha[-3])
            except:
                try:
                    gu(sha[-2])
                except:
                    try:
                        gu(sha[-1])
                    except Exception as e:
                        print(location, "; url didn't go through")
                        print(e)
                        return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    ticket_list=''
    org_name='The International Academic Forum (IAFOR)'
    org_web='https://iafor.org/'
    org_pro='Since 2009, The International Academic Forum (IAFOR) has welcomed over 40,000 academics to its interdisciplinary conferences held around the world. IAFOR has hosted conferences in Tokyo, Osaka, Kobe and Kyoto (Japan), London and Brighton (UK), Barcelona (Spain), New York, Virginia, Rhode Island and Hawaii (USA), Hong Kong (Hong Kong SAR), Singapore, and Dubai (UAE).'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    time_=''
    event_info=''
    Speakerlist=''
    category=''
    mail_=''
    event_web=link
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[-2]+', '+loc.split(',')[-1].strip()
            country=loc.split(',')[-1].strip()
        else:
            city=loc
            country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)

    iafor.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])

print('Done')

Done


In [23]:
len(iafor)

0

In [21]:
iafor

[]

In [24]:
iafor_dict=iafor


iafor_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=iafor_dict)
iafor_df.to_csv("iafor.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [7]:
# s=1
# for k in sanp:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[1])
#     s+=1
# print(s)

17


In [12]:
# driver.switch_to.window(driver.window_handles[0])

In [13]:
# truce=[]
# for k in range(1,22):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
    
#     try:
#         info=' '.join(soupn.find('div',{'class':'my-6 text-lg leading-relaxed'}).text.split())
#     except:
#         info=' '.join(soupn.find('div',{'class':'wysiwyg-content'}).findChild().text.split())
#         if ':' in info:
#             info='.'.join(info.split('.')[:-1])+'.'
#         if 'Click here' in info:
#             info=info.split('Click here')[0].strip()
#     #**************************
#     try:
#         mam=' '.join(soupn.find('div',{'class':'my-12'}).get_text(separator='}').split())
#         tida=mam.split('} }')[0].replace('–','-')
#         strt,enrt=tida.split('}')
#         das,tis=strt.split('|')
#         dae,tie=enrt.split('|')
#         da=das+'-'+dae
#         ti=tis+'-'+tie
#     except:
#         sham=soupn.find('div',{'class':'filterable-list--simple-two'}).text
#         diy=soupn.find_all('div',{'class':'filterable-list__row'})
#         if 'Time:' in sham:
#             for o in diy:
#                 if 'Time:' in str(o):
#                     ti=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split()).strip()
#                     if '|' in ti:
#                         try:
#                             ti=ti.split('|')[0].split('):')[1].strip()
#                         except:
#                             ti=ti.split('|')[0].strip()
#                 elif 'Date:' in str(o):
#                     da=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split()).strip()
#                 #tida=ti+'|'+da
#         else:
#             for o in diy:
#                 if 'Date:' in str(o):
#                     tida=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split())
            
#             if '|' in tida:

#                 ii=tida.split('|')
#                 da=ii[0].strip()
#                 if '-' in da:
        
#                     da=da.split('-')[0].strip()
        
#                 if ';' in ii[1]:
#                     ti=ii[1].split(';')[0].strip()
#                 else:
#                     ti=ii[1].split(',')[0].strip()
#             else:
#                 ii=tida.split('-')
#                 da=ii[0].strip()
#                 if ';' in ii[1]:
#                     ti=ii[1].split(';')[0].strip()
#                 else:
#                     ti=ii[1].split(',')[0].strip()
#     for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
#         if f in da:
#             da=da.replace(f,'').replace(',','').strip()
#     dat=da.replace(',','').replace('.','').replace('  ',' ').replace('Sept','September').strip()
#     if '-' in dat:
#         st=dat.split('-')[0].strip()
#         en=dat.split('-')[1].strip()
#     elif '/' in dat:
#         dg=dat.split(' ')
#         st,en=dg[0],dg[-1]
#     else:
#         st=en=dat
#     #change ti format
#     ti=ti.replace('.','').upper().replace(' AM','AM').replace(' PM','PM')

#     #
#     if '-' in ti:
#         ra=ti.split('-')[0].strip()
#         ma=ti.split('-')[1].strip()
#         try:
#             z=ma.split(' ')[1]
#             da=ma.split(' ')[0]
#         except:
#             z=''
#             da=ma
#     else:
#         ra=ti.split(' ')[0].strip()
#         da=''
#         z=ti.split(' ')[1].strip()
# #     if 'AM' in ra or 'PM' in ra:
# #         rt=datetime.strptime(da, '%H:%M')
# #         da=rt.strftime('%I:%M%p')
# #         mt=datetime.strptime(ra, '%H:%M')
# #         ra=mt.strftime('%I:%M%p')
#     time_st={'type':'general',
#              'Start_time':ra,
#              'end_time':da,
#              'timezone':z,
#              'days':'all'

#     }
#     time_=json.dumps([time_st])
#     time_
    
    
    
#     #                        02        April      2022
#     if re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#      #   print('awujale')
#         start=st
#         end=en
#     #special case 5/26
#     elif '/' in st:
#         if len(st.split('/'))==2:
#             yr=str(datetime.today().year)
#             br=st+' '+yr
#             mo=datetime.strptime(br, '%m/%d %Y')
#             start=end=str(mo.strftime('%d %B %Y'))
#             if '/' not in en:  
#                 start=end
#             else:
#                 tr=en+' '+yr
#                 mo=datetime.strptime(tr, '%m/%d %Y')
#                 end=str(mo.strftime('%d %B %Y'))
        

#     #*                        April       02      2022
#     elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#     #       print('ala')
#         pa=re.search('(\d{4})',st).group()
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()
#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba     

#     #                      02     2022
#     elif re.search('\d{1,2}\s+\d{4}',en):
#     #      print('kum')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',st).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     #                      02         April
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#     #        print('is')
#         pa=re.search('(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en

#     #*                      April           02
#     elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#     #      print('bad')
#         try:
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         except:
#             pa=str(datetime.today().year)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         try:
#             ba=re.search('(\d{4})',en).group()
#         except:
#             ba=str(datetime.today().year)
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba
#     #**
#     if start=='':
#         start_date=end_date=''
#     else:
#         pick=[start, end]
#         try:
#             spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
#         except:
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#         date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#         start_date=date_[0]
#         end_date=date_[1]

#     truce.append([time_,start_date,end_date,info])
# truce